In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ta import add_all_ta_features
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load stock market data
def load_stock_data(ticker, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date)

# Add technical indicators
def add_technical_indicators(df):
    return add_all_ta_features(
        df, open="Open", high="High", low="Low", close="Close", volume="Volume")

# Predictive Model: Random Forest Regressor
def predict_stock_prices(df, feature_columns, target_column):
    X = df[feature_columns]
    y = df[target_column].shift(-1)  # Predict next day's close price
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    model = RandomForestRegressor(n_estimators=100)
    model.fit(X_train, y_train[:-1])  # Exclude last NaN
    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test[:-1], predictions)
    print(f"Mean Squared Error: {mse}")

    return predictions

# Automated Alert System
def automated_alerts(df, threshold=0.05):
    """
    Send an automated alert if the stock price changes significantly
    :param df: DataFrame with stock prices
    :param threshold: Threshold for sending an alert (percentage change)
    """
    previous_day_price = df['Close'].shift(1)
    change = df['Close'] - previous_day_price
    percent_change = change / previous_day_price

    alerts = percent_change.abs() > threshold
    alert_dates = df[alerts].index
    return alert_dates

# Main execution
if __name__ == '__main__':
    data = load_stock_data('AAPL', '2020-01-01', '2021-01-01')
    data_with_indicators = add_technical_indicators(data)

    # Predict future stock prices
    feature_cols = ['Volume', 'momentum_rsi', 'trend_macd', 'volatility_bbh', 'volatility_bbl']
    predictions = predict_stock_prices(data_with_indicators, feature_cols, 'Close')

    # Automated alerts for significant price changes
    alert_dates = automated_alerts(data_with_indicators)
    print(f"Alert Dates for significant price changes: {alert_dates}")

    # Interactive plot (optional)
    plotly_figure = px.line(data_with_indicators, x=data_with_indicators.index, y='Close', title='AAPL Stock Prices')
    plotly_figure.show()
